In [0]:
# pyspark libraries
import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.sql import Window

from functools import reduce

import databricks.koalas as ks
import pandas as pd
import numpy as np

import re
import io
import datetime

# plotting libraries
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
import seaborn as sns


# versions
print("Matplotlib version: ", matplotlib.__version__)
print("Seaborn version: ", sns.__version__)
_datetimenow = datetime.datetime.now() # .strftime("%Y%m%d")
print(f"_datetimenow:  {_datetimenow}")

In [0]:
%run "/Repos/shds/common/functions"

In [0]:
%run "/Repos/shds/Fionna/help_functions"

In [0]:
%run "./CCU056-01-parameters"

In [0]:
bhf_phenotypes = spark.table(path_ref_bhf_phenotypes)

In [0]:
display(bhf_phenotypes)

In [0]:
display(bhf_phenotypes.filter(f.col("name")=="hypercholesterolaemia"))

#1. Comorbidities

##1.1 Heart Failure

In [0]:
comorbidities_hf = (bhf_phenotypes.filter(f.col("name")=="HF").select("name","terminology","code","term"))

display(comorbidities_hf)

##1.2 COPD

In [0]:
comorbidities_copd = (bhf_phenotypes.filter(f.col("name")=="COPD").select("name","terminology","code","term"))

#from CCU004_01
codelist_copd = spark.createDataFrame(
[
("copd","SNOMED","106001000119101","chronic obstructive lung disease co-occurrent with acute bronchitis (disorder)","1","20220215"),
("copd","SNOMED","135836000","end stage chronic obstructive airways disease (disorder)","1","20220215"),
("copd","SNOMED","13645005","chronic obstructive lung disease (disorder)","1","20220215"),
("copd","SNOMED","16003001","giant bullous emphysema (disorder)","1","20220215"),
("copd","SNOMED","16846004","obstructive emphysema (disorder)","1","20220215"),
("copd","SNOMED","185086009","chronic obstructive bronchitis (disorder)","1","20220215"),
("copd","SNOMED","195951007","acute exacerbation of chronic obstructive airways disease (disorder)","1","20220215"),
("copd","SNOMED","195958001","segmental bullous emphysema (disorder)","1","20220215"),
("copd","SNOMED","195959009","zonal bullous emphysema (disorder)","1","20220215"),
("copd","SNOMED","196001008","chronic obstructive pulmonary disease with acute lower respiratory infection (disorder)","1","20220215"),
("copd","SNOMED","233674008","pulmonary emphysema in alpha-1 primary immunodeficiency deficiency (disorder)","1","20220215"),
("copd","SNOMED","266355005","bullous emphysema with collapse (disorder)","1","20220215"),
("copd","SNOMED","285381006","acute infective exacerbation of chronic obstructive airways disease (disorder)","1","20220215"),
("copd","SNOMED","293241000119100","acute exacerbation of chronic obstructive bronchitis (disorder)","1","20220215"),
("copd","SNOMED","293991000000106","very severe chronic obstructive pulmonary disease (disorder)","1","20220215"),
("copd","SNOMED","313296004","mild chronic obstructive pulmonary disease (disorder)","1","20220215"),
("copd","SNOMED","313297008","moderate chronic obstructive pulmonary disease (disorder)","1","20220215"),
("copd","SNOMED","313299006","severe chronic obstructive pulmonary disease (disorder)","1","20220215"),
("copd","SNOMED","31898008","paraseptal emphysema (disorder)","1","20220215"),
("copd","SNOMED","45145000","unilateral emphysema (situation)","1","20220215"),
("copd","SNOMED","4981000","panacinar emphysema (disorder)","1","20220215"),
("copd","SNOMED","66110007","chronic diffuse emphysema caused by inhalation of chemical fumes and/or vapors (disorder)","1","20220215"),
("copd","SNOMED","68328006","centriacinar emphysema (disorder)","1","20220215"),
("copd","SNOMED","847091000000104","acute non-infective exacerbation of chronic obstructive pulmonary disease (disorder)","1","20220215"),
("copd","SNOMED","87433001","pulmonary emphysema (disorder)","1","20220215"),
("copd","SNOMED","408501008","emergency hospital admission for chronic obstructive pulmonary disease (procedure)","1","20220215"),
("copd","SNOMED","1110861000000102","quality and outcomes framework chronic obstructive pulmonary disease quality indicator-related care invitation (procedure)","1","20220215"),
("copd","SNOMED","143371000000104","quality and outcomes framework chronic obstructive pulmonary disease quality indicator-related care invitation using preferred method of communication (procedure)","1","20220215"),
("copd","SNOMED","836477007","chronic emphysema caused by vapor (disorder)","1","20220215"),
("copd","SNOMED","1010333003","emphysema of left lung (disorder)","1","20220215"),
("copd","SNOMED","1010334009","emphysema of right lung (disorder)","1","20220215"), 

# additional codes to include (ICD10)- fc
("copd","ICD10","J41","Simple and mucopurulent chronic bronchitis",'',''), #removed in proj CCU052
("copd","ICD10","J410","Simple chronic bronchitis",'',''),
("copd","ICD10","J411","Mucopurulent chronic bronchitis",'',''), #removed in proj CCU052
("copd","ICD10","J418","Mixed simple and mucopurulent chronic bronchitis",'',''), #removed in proj CCU052
("copd","ICD10","J42X","Unspecified chronic bronchitis",'',''),
("copd","ICD10","J43","Emphysema",'',''),
("copd","ICD10","J430","MacLeod syndrome",'',''),
("copd","ICD10","J431","Panlobular emphysema",'',''),
("copd","ICD10","J432","Centrilobular emphysema",'',''),
("copd","ICD10","J438","Other emphysema",'',''),
("copd","ICD10","J439","Emphysema, unspecified",'',''),
("copd","ICD10","J44","Other chronic obstructive pulmonary disease",'',''),
("copd","ICD10","J440","Chronic obstructive pulmonary disease with acute lower respiratory infection",'',''),
("copd","ICD10","J441","Chronic obstructive pulmonary disease with acute exacerbation, unspecified",'',''),
("copd","ICD10","J448","Other specified chronic obstructive pulmonary disease",'',''),
("copd","ICD10","J449","Chronic obstructive pulmonary disease, unspecified",'','')
],
['name', 'terminology', 'code', 'term', 'code_type', 'RecordDate']   
).select("name","terminology","code","term")


comorbidities_copd = comorbidities_copd.union(codelist_copd).distinct().withColumn("name", f.when(f.col("name") == "COPD", "copd").otherwise(f.col("name")))

display(comorbidities_copd)

##1.3 Asthma

In [0]:
#from CCU004_01
codelist_asthma = spark.createDataFrame(
[
("asthma","SNOMED","1064811000000103","moderate acute exacerbation of asthma (disorder)","1","20220215"),
("asthma","SNOMED","1064821000000109","life threatening acute exacerbation of asthma (disorder)","1","20220215"),
("asthma","SNOMED","10674711000119105","acute severe exacerbation of asthma co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","10675391000119101","severe controlled persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","10675431000119106","severe persistent allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","10675471000119109","acute severe exacerbation of severe persistent allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","10675551000119104","acute severe exacerbation of severe persistent asthma co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","10675591000119109","severe persistent allergic asthma controlled (finding)","1","20220215"),
("asthma","SNOMED","10675631000119109","severe persistent asthma controlled co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","10675671000119107","severe persistent allergic asthma uncontrolled (finding)","1","20220215"),
("asthma","SNOMED","10675711000119106","severe persistent asthma uncontrolled co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","10675751000119107","severe uncontrolled persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","10675871000119106","mild persistent allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","10675911000119109","acute severe exacerbation of mild persistent allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","10675991000119100","acute severe exacerbation of mild persistent allergic asthma co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","10676031000119106","mild persistent allergic asthma controlled (finding)","1","20220215"),
("asthma","SNOMED","10676071000119109","mild persistent asthma controlled (finding)","1","20220215"),
("asthma","SNOMED","10676111000119102","mild persistent asthma controlled co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","10676151000119101","mild persistent allergic asthma uncontrolled (finding)","1","20220215"),
("asthma","SNOMED","10676191000119106","mild persistent asthma uncontrolled (finding)","1","20220215"),
("asthma","SNOMED","10676231000119102","mild persistent asthma uncontrolled co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","10676351000119103","moderate persistent asthma controlled (finding)","1","20220215"),
("asthma","SNOMED","10676391000119108","moderate persistent allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","10676431000119103","acute severe exacerbation of moderate persistent allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","10676511000119109","acute severe exacerbation of moderate persistent asthma co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","10676551000119105","moderate persistent allergic asthma controlled (finding)","1","20220215"),
("asthma","SNOMED","10676591000119100","moderate persistent controlled asthma co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","10676631000119100","moderate persistent allergic asthma uncontrolled (finding)","1","20220215"),
("asthma","SNOMED","10676671000119102","moderate persistent asthma uncontrolled co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","10676711000119103","moderate persistent asthma uncontrolled (finding)","1","20220215"),
("asthma","SNOMED","10692721000119102","chronic obstructive asthma co-occurrent with acute exacerbation of asthma (disorder)","1","20220215"),
("asthma","SNOMED","1086701000000102","life threatening acute exacerbation of allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","1086711000000100","life threatening acute exacerbation of intrinsic asthma (disorder)","1","20220215"),
("asthma","SNOMED","1103911000000103","severe asthma with fungal sensitisation (disorder)","1","20220215"),
("asthma","SNOMED","11641008","millers' asthma (disorder)","1","20220215"),
("asthma","SNOMED","12428000","intrinsic asthma without status asthmaticus (disorder)","1","20220215"),
("asthma","SNOMED","124991000119109","severe persistent asthma co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","125001000119103","moderate persistent asthma co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","125011000119100","mild persistent asthma co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","125021000119107","intermittent asthma co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","135171000119106","acute exacerbation of moderate persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","135181000119109","acute exacerbation of mild persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","16584951000119101","oral steroid-dependent asthma (disorder)","1","20220215"),
("asthma","SNOMED","1741000119102","intermittent asthma uncontrolled (disorder)","1","20220215"),
("asthma","SNOMED","1751000119100","acute exacerbation of chronic obstructive airways disease with asthma (disorder)","1","20220215"),
("asthma","SNOMED","18041002","printers' asthma (disorder)","1","20220215"),
("asthma","SNOMED","195967001","asthma (disorder)","1","20220215"),
("asthma","SNOMED","195977004","mixed asthma (disorder)","1","20220215"),
("asthma","SNOMED","225057002","brittle asthma (disorder)","1","20220215"),
("asthma","SNOMED","233679003","late onset asthma (disorder)","1","20220215"),
("asthma","SNOMED","233683003","hay fever with asthma (disorder)","1","20220215"),
("asthma","SNOMED","233688007","sulfite-induced asthma (disorder)","1","20220215"),
("asthma","SNOMED","233691007","asthmatic pulmonary eosinophilia (disorder)","1","20220215"),
("asthma","SNOMED","2360001000004109","steroid dependent asthma (disorder)","1","20220215"),
("asthma","SNOMED","266361008","non-allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","281239006","exacerbation of asthma (disorder)","1","20220215"),
("asthma","SNOMED","30352005","allergic-infective asthma (disorder)","1","20220215"),
("asthma","SNOMED","304527002","acute asthma (disorder)","1","20220215"),
("asthma","SNOMED","312453004","asthma - currently active (finding)","1","20220215"),
("asthma","SNOMED","31387002","exercise-induced asthma (disorder)","1","20220215"),
("asthma","SNOMED","34015007","bakers' asthma (disorder)","1","20220215"),
("asthma","SNOMED","370218001","mild asthma (disorder)","1","20220215"),
("asthma","SNOMED","370219009","moderate asthma (disorder)","1","20220215"),
("asthma","SNOMED","370220003","occasional asthma (disorder)","1","20220215"),
("asthma","SNOMED","370221004","severe asthma (disorder)","1","20220215"),
("asthma","SNOMED","389145006","allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","395022009","asthma night-time symptoms (finding)","1","20220215"),
("asthma","SNOMED","401000119107","asthma with irreversible airway obstruction (disorder)","1","20220215"),
("asthma","SNOMED","401193004","asthma confirmed (situation)","1","20220215"),
("asthma","SNOMED","404804003","platinum asthma (disorder)","1","20220215"),
("asthma","SNOMED","404806001","cheese-makers' asthma (disorder)","1","20220215"),
("asthma","SNOMED","404808000","isocyanate induced asthma (disorder)","1","20220215"),
("asthma","SNOMED","405944004","asthmatic bronchitis (disorder)","1","20220215"),
("asthma","SNOMED","407674008","aspirin-induced asthma (disorder)","1","20220215"),
("asthma","SNOMED","409663006","cough variant asthma (disorder)","1","20220215"),
("asthma","SNOMED","41553006","detergent asthma (disorder)","1","20220215"),
("asthma","SNOMED","418395004","tea-makers' asthma (disorder)","1","20220215"),
("asthma","SNOMED","423889005","non-immunoglobulin e mediated allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","424199006","substance induced asthma (disorder)","1","20220215"),
("asthma","SNOMED","424643009","immunoglobulin e-mediated allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","425969006","exacerbation of intermittent asthma (disorder)","1","20220215"),
("asthma","SNOMED","426656000","severe persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","426979002","mild persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","427295004","moderate persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","427603009","intermittent asthma (disorder)","1","20220215"),
("asthma","SNOMED","427679007","mild intermittent asthma (disorder)","1","20220215"),
("asthma","SNOMED","442025000","acute exacerbation of chronic asthmatic bronchitis (disorder)","1","20220215"),
("asthma","SNOMED","445427006","seasonal asthma (disorder)","1","20220215"),
("asthma","SNOMED","55570000","asthma without status asthmaticus (disorder)","1","20220215"),
("asthma","SNOMED","56968009","asthma caused by wood dust (disorder)","1","20220215"),
("asthma","SNOMED","57607007","occupational asthma (disorder)","1","20220215"),
("asthma","SNOMED","63088003","allergic asthma without status asthmaticus (disorder)","1","20220215"),
("asthma","SNOMED","641000119106","intermittent asthma well controlled (finding)","1","20220215"),
("asthma","SNOMED","703953004","allergic asthma caused by dermatophagoides pteronyssinus (disorder)","1","20220215"),
("asthma","SNOMED","703954005","allergic asthma caused by dermatophagoides farinae (disorder)","1","20220215"),
("asthma","SNOMED","707444001","uncomplicated asthma (disorder)","1","20220215"),
("asthma","SNOMED","707445000","exacerbation of mild persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","707446004","exacerbation of moderate persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","707447008","exacerbation of severe persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","707511009","uncomplicated mild persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","707512002","uncomplicated moderate persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","707513007","uncomplicated severe persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","707979007","acute severe exacerbation of severe persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","707980005","acute severe exacerbation of moderate persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","707981009","acute severe exacerbation of mild persistent asthma (disorder)","1","20220215"),
("asthma","SNOMED","708038006","acute exacerbation of asthma (disorder)","1","20220215"),
("asthma","SNOMED","708090002","acute severe exacerbation of asthma (disorder)","1","20220215"),
("asthma","SNOMED","708093000","acute exacerbation of immunoglobulin e-mediated allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","708094006","acute exacerbation of intrinsic asthma (disorder)","1","20220215"),
("asthma","SNOMED","708095007","acute severe exacerbation of immunoglobin e-mediated allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","708096008","acute severe exacerbation of intrinsic asthma (disorder)","1","20220215"),
("asthma","SNOMED","72301000119103","asthma in pregnancy (disorder)","1","20220215"),
("asthma","SNOMED","733858005","acute severe refractory exacerbation of asthma (disorder)","1","20220215"),
("asthma","SNOMED","734904007","life threatening acute exacerbation of asthma (disorder)","1","20220215"),
("asthma","SNOMED","734905008","moderate acute exacerbation of asthma (disorder)","1","20220215"),
("asthma","SNOMED","735587000","acute severe exacerbation of asthma co-occurrent and due to allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","735588005","uncomplicated allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","735589002","uncomplicated non-allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","762521001","exacerbation of allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","782513000","acute severe exacerbation of allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","782520007","exacerbation of allergic asthma due to infection (disorder)","1","20220215"),
("asthma","SNOMED","786836003","near fatal asthma (disorder)","1","20220215"),
("asthma","SNOMED","866881000000101","chronic asthma with fixed airflow obstruction (disorder)","1","20220215"),
("asthma","SNOMED","92807009","chemical-induced asthma (disorder)","1","20220215"),
("asthma","SNOMED","93432008","drug-induced asthma (disorder)","1","20220215"),
("asthma","SNOMED","99031000119107","acute exacerbation of asthma co-occurrent with allergic rhinitis (disorder)","1","20220215"),
("asthma","SNOMED","183478001","emergency hospital admission for asthma (procedure)","1","20220215"),
("asthma","SNOMED","708358003","emergency asthma admission since last encounter (situation)","1","20220215"),
("asthma","SNOMED","170631002","asthma disturbing sleep (finding)","1","20220215"),
("asthma","SNOMED","170632009","asthma causing night waking (finding)","1","20220215"),
("asthma","SNOMED","170633004","asthma disturbs sleep weekly (finding)","1","20220215"),
("asthma","SNOMED","170634005","asthma disturbs sleep frequently (finding)","1","20220215"),
("asthma","SNOMED","170635006","asthma not disturbing sleep (finding)","1","20220215"),
("asthma","SNOMED","170636007","asthma never disturbs sleep (finding)","1","20220215"),
("asthma","SNOMED","170637003","asthma limiting activities (finding)","1","20220215"),
("asthma","SNOMED","170638008","asthma not limiting activities (finding)","1","20220215"),
("asthma","SNOMED","170655007","asthma restricts exercise (finding)","1","20220215"),
("asthma","SNOMED","170656008","asthma sometimes restricts exercise (finding)","1","20220215"),
("asthma","SNOMED","170657004","asthma severely restricts exercise (finding)","1","20220215"),
("asthma","SNOMED","170658009","asthma never restricts exercise (finding)","1","20220215"),
("asthma","SNOMED","270442000","asthma monitoring check done (finding)","1","20220215"),
("asthma","SNOMED","370202007","asthma causes daytime symptoms 1 to 2 times per month (finding)","1","20220215"),
("asthma","SNOMED","370203002","asthma causes daytime symptoms 1 to 2 times per week (finding)","1","20220215"),
("asthma","SNOMED","370204008","asthma causes daytime symptoms most days (finding)","1","20220215"),
("asthma","SNOMED","370205009","asthma causes night symptoms 1 to 2 times per month (finding)","1","20220215"),
("asthma","SNOMED","370206005","asthma limits walking on the flat (finding)","1","20220215"),
("asthma","SNOMED","370207001","asthma limits walking up hills or stairs (finding)","1","20220215"),
("asthma","SNOMED","370208006","asthma never causes daytime symptoms (finding)","1","20220215"),
("asthma","SNOMED","373899003","asthma daytime symptoms (finding)","1","20220215"),
("asthma","SNOMED","390872009","change in asthma management plan (regime/therapy)","1","20220215"),
("asthma","SNOMED","390877003","step up change in asthma management plan (regime/therapy)","1","20220215"),
("asthma","SNOMED","390878008","step down change in asthma management plan (regime/therapy)","1","20220215"),
("asthma","SNOMED","390921001","absent from work or school due to asthma (finding)","1","20220215"),
("asthma","SNOMED","394700004","asthma annual review (regime/therapy)","1","20220215"),
("asthma","SNOMED","473391009","asthma never causes night symptoms (situation)","1","20220215"),
("asthma","SNOMED","771901000000100","asthma causes night time symptoms 1 to 2 times per week (finding)","1","20220215"),
("asthma","SNOMED","771941000000102","asthma causes symptoms most nights (finding)","1","20220215"),
("asthma","SNOMED","771981000000105","asthma limits activities 1 to 2 times per month (finding)","1","20220215"),
("asthma","SNOMED","772011000000107","asthma limits activities 1 to 2 times per week (finding)","1","20220215"),
("asthma","SNOMED","772051000000106","asthma limits activities most days (finding)","1","20220215"),
("asthma","SNOMED","811151000000105","number of days absent from school due to asthma in past 6 months (observable entity)","1","20220215"),
("asthma","SNOMED","10674991000119104","intermittent allergic asthma (disorder)","1","20220215"),
("asthma","SNOMED","829976001","thunderstorm asthma (disorder)","1","20220215"),
("asthma","SNOMED","782559003","asthma never causes night symptoms (finding)","1","20220215"),

# additional codes to include (ICD10)- fc
("asthma","ICD10","J450","Predominantly allergic asthma",'',''),
("asthma","ICD10","J451","Nonallergic asthma",'',''),
("asthma","ICD10","J458","Mixed asthma",'',''),
("asthma","ICD10","J459","Asthma, unspecifieda",'',''),
("asthma","ICD10","J46X","Status asthmaticusa",'','')
  
],
['name', 'terminology', 'code', 'term', 'code_type', 'RecordDate']   
).select("name","terminology","code","term")


comorbidities_asthma = codelist_asthma.distinct()

display(comorbidities_asthma)

##1.4 Renal Failure (using CKD)

In [0]:
comorbidities_ckd = (bhf_phenotypes.filter(f.col("name")=="CKD").select("name","terminology","code","term"))

display(comorbidities_ckd)

##1.5 Cerebrovascular Disease

Include all stroke groups and Cerebrovascular accidents (CVA). CVAs are included in the stroke codes e.g. SNOMED 230690007

In [0]:
comorbidities_cd = (bhf_phenotypes.filter(f.col("name").isin("stroke_HS", "stroke_IS", "stroke_NOS", "stroke_SAH", "stroke_TIA"))
                    .select("name","terminology","code","term"))

display(comorbidities_cd)

##1.6 Peripheral Arterial Disease

In [0]:
comorbidities_pad_bhf = (bhf_phenotypes.filter(f.col("name").isin("pad"))
                    .select("name","terminology","code","term"))

# ICD10 added by Fionna on 23/11/2023 - sourced form https://datacompass.lshtm.ac.uk/id/eprint/3259/
# Note that many of these ICD10 codes are in the Outcomes - CVD Death list under atherosclerosis
# Why have I added ICD10 codes? Currently the comorbidites and risk factors prevalences are increasing year on year in the 2000-2023 cohort - this is possibly (likely) because people are only in GDPPR if they were alive as at 2019 thus comorbidities may have to be derived only using secondary care. This will mean that each is underestimated but this underestimation will be consistent across groups and by year.

comorbidities_pad_icd10 = (spark.createDataFrame(
  [
    ('pad','ICD10',  'I702',   'Atherosclerosis of arteries of extremities','',''),   
    ('pad','ICD10',  'I7020',   'Atherosclerosis of arteries of extremities','',''),
    ('pad','ICD10',  'I7021',   'Atherosclerosis of arteries of extremities','',''),

    ('pad','ICD10',  'I73',   'Other peripheral vascular diseases','',''), 
    ('pad','ICD10',  'I738',   'Other specified peripheral vascular diseases','',''),
    ('pad','ICD10',  'I739',   'Peripheral vascular disease, unspecified','',''), 

    ('pad','ICD10',  'I792',   'Peripheral angiopathy in diseases classified elsewhere','','')
  ],
  ['name', 'terminology', 'code', 'term', 'code_type', 'RecordDate']  
).select("name","terminology","code","term")
)

comorbidities_pad = comorbidities_pad_bhf.union(comorbidities_pad_icd10)


display(comorbidities_pad)

#2. Cardiovascular Risk Factors

##2.1 Hypertension

In [0]:
crf_hypertension_bhf = (bhf_phenotypes.filter(f.col("name").isin("hypertension"))
                    .select("name","terminology","code","term"))


crf_hypertension_icd10 = (spark.createDataFrame(
  [
    #hypertensive disease
    ('hypertension','ICD10',  'I10',   'Essential (primary) hypertension','',''),
    ('hypertension','ICD10',  'I11',   'Hypertensive heart disease','',''),
    ('hypertension','ICD10',  'I110',   'Hypertensive heart disease with (congestive) heart failure','',''),
    ('hypertension','ICD10',  'I119',   'Hypertensive heart disease without (congestive) heart failure','',''),
    ('hypertension','ICD10',  'I12',   'Hypertensive renal disease','',''),
    ('hypertension','ICD10',  'I129',   'Hypertensive renal disease without renal failure','',''),
    ('hypertension','ICD10',  'I13',   'Hypertensive heart and renal disease','',''),
    ('hypertension','ICD10',  'I130',   'Hypertensive heart and renal disease with (congestive) heart failure','',''),
    ('hypertension','ICD10',  'I131',   'Hypertensive heart and renal disease with renal failure','',''),
    ('hypertension','ICD10',  'I132',   'Hypertensive heart and renal disease with both (congestive) heart failure and renal failure','',''),
    ('hypertension','ICD10',  'I139',   'Hypertensive heart and renal disease, unspecified','',''),
    ('hypertension','ICD10',  'I15',   'Secondary hypertension','',''),
    ('hypertension','ICD10',  'I151',   'Hypertension secondary to other renal disorders','',''),
    ('hypertension','ICD10',  'I152',   'Hypertension secondary to endocrine disorders','',''),
    ('hypertension','ICD10',  'I158',   'Other secondary hypertension','',''),
    ('hypertension','ICD10',  'I159',   'Secondary hypertension, unspecified','','')
  ],
  ['name', 'terminology', 'code', 'term', 'code_type', 'RecordDate']  
).select("name","terminology","code","term")
)

crf_hypertension = crf_hypertension_bhf.union(crf_hypertension_icd10)

display(crf_hypertension)

##2.2 Diabetes

In [0]:
crf_diabetes_bhf = (bhf_phenotypes.filter(f.col("name").isin("diabetes"))
                    .select("name","terminology","code","term"))

# ICD10 codes added by Fionna on 23/11/2023 - copied from CCU000 from the Phenotype Library

crf_diabetes_icd10 = (spark.createDataFrame(
  [

    ('diabetes','ICD10',  'E10',   'Insulin-dependent diabetes mellitus','',''),
    ('diabetes','ICD10',  'E11',   'Non-insulin-dependent diabetes mellitus','',''),
    ('diabetes','ICD10',  'E12',   'Malnutrition-related diabetes mellitus','',''),
    ('diabetes','ICD10',  'E13',   'Other specified diabetes mellitus','',''),
    ('diabetes','ICD10',  'E14',   'Unspecified diabetes mellitus','',''),
    ('diabetes','ICD10',  'G590',   'Diabetic mononeuropathy','',''),
    ('diabetes','ICD10',  'G632',   'Diabetic polyneuropathy','',''),
    ('diabetes','ICD10',  'H280',   'Diabetic cataract','',''),
    ('diabetes','ICD10',  'H360',   'Diabetic retinopathy','',''),
    ('diabetes','ICD10',  'M142',   'Diabetic arthropathy','',''),
    ('diabetes','ICD10',  'N083',   'Glomerular disorders in diabetes mellitus','',''),
    ('diabetes','ICD10',  'O240',   'Diabetes mellitus in pregnancy: Pre-existing diabetes mellitus, insulin-dependent','',''),
    ('diabetes','ICD10',  'O241',   'Diabetes mellitus in pregnancy: Pre-existing diabetes mellitus, non-insulin-dependent','',''),
    ('diabetes','ICD10',  'O242',   'Diabetes mellitus in pregnancy: Pre-existing malnutrition-related diabetes mellitus','',''),
    ('diabetes','ICD10',  'O243',   'Diabetes mellitus in pregnancy: Pre-existing diabetes mellitus, unspecified','','')

  ],
  ['name', 'terminology', 'code', 'term', 'code_type', 'RecordDate']  
).select("name","terminology","code","term")
)

crf_diabetes = crf_diabetes_bhf.union(crf_diabetes_icd10)


display(crf_diabetes)

##2.3 Coronary Artery Disease

E.g. Ischaemic Heart Disease (dont think there is a general code for this) incldues:
- MI (AMI BHF Phenotype Library)
- Angina and Unstable Angina from CCU002_01

In [0]:
crf_cad = (bhf_phenotypes.filter(f.col("name").isin("AMI"))
                    .select("name","terminology","code","term"))



# from CCU002_01_codelist
codelist_angina = spark.createDataFrame(
[
("angina","ICD10","I20","Angina","",""),
("angina","SNOMED","10971000087107","Myocardial ischemia during surgery (disorder)","1","20210127"),
("angina","SNOMED","15960061000119102","Unstable angina co-occurrent and due to coronary arteriosclerosis (disorder)","1","20210127"),
("angina","SNOMED","15960581000119102","Angina co-occurrent and due to arteriosclerosis of autologous vein coronary artery bypass graft (disorder)","1","20210127"),
("angina","SNOMED","15960661000119107","Unstable angina co-occurrent and due to arteriosclerosis of coronary artery bypass graft (disorder)","1","20210127"),
("angina","SNOMED","19057007","Status anginosus (disorder)","1","20210127"),
("angina","SNOMED","194828000","Angina (disorder)","1","20210127"),
("angina","SNOMED","21470009","Syncope anginosa (disorder)","1","20210127"),
("angina","SNOMED","233821000","New onset angina (disorder)","1","20210127"),
("angina","SNOMED","25106000","Impending infarction (disorder)","1","20210127"),
("angina","SNOMED","314116003","Post infarct angina (disorder)","1","20210127"),
("angina","SNOMED","371806006","Progressive angina (disorder)","1","20210127"),
("angina","SNOMED","371809004","Recurrent angina status post coronary stent placement (disorder)","1","20210127"),
("angina","SNOMED","371812001","Recurrent angina status post directional coronary atherectomy (disorder)","1","20210127"),
("angina","SNOMED","41334000","Angina, class II (disorder)","1","20210127"),
("angina","SNOMED","413439005","Acute ischemic heart disease (disorder)","1","20210127"),
("angina","SNOMED","413444003","Acute myocardial ischemia (disorder)","1","20210127"),
("angina","SNOMED","413838009","Chronic ischemic heart disease (disorder)","1","20210127"),
("angina","SNOMED","429559004","Typical angina (disorder)","1","20210127"),
("angina","SNOMED","4557003","Preinfarction syndrome (disorder)","1","20210127"),
("angina","SNOMED","59021001","Angina decubitus (disorder)","1","20210127"),
("angina","SNOMED","61490001","Angina, class I (disorder)","1","20210127"),
("angina","SNOMED","712866001","Resting ischemia co-occurrent and due to ischemic heart disease (disorder)","1","20210127"),
("angina","SNOMED","791000119109","Angina associated with type II diabetes mellitus (disorder)","1","20210127"),
("angina","SNOMED","85284003","Angina, class III (disorder)","1","20210127"),
("angina","SNOMED","89323001","Angina, class IV (disorder)","1","20210127"),
("angina","SNOMED","15960141000119102","Angina co-occurrent and due to coronary arteriosclerosis (disorder)","1","20210127"),
("angina","SNOMED","15960381000119109","Angina co-occurrent and due to arteriosclerosis of coronary artery bypass graft (disorder)","1","20210127"),
("angina","SNOMED","194823009","Acute coronary insufficiency (disorder)","1","20210127"),
("angina","SNOMED","225566008","Ischemic chest pain (finding)","1","20210127"),
("angina","SNOMED","233819005","Stable angina (disorder)","1","20210127"),
("angina","SNOMED","233823002","Silent myocardial ischemia (disorder)","1","20210127"),
("angina","SNOMED","300995000","Exercise-induced angina (disorder)","1","20210127"),
("angina","SNOMED","315025001","Refractory angina (disorder)","1","20210127"),
("angina","SNOMED","35928006","Nocturnal angina (disorder)","1","20210127"),
("angina","SNOMED","371807002","Atypical angina (disorder)","1","20210127"),
("angina","SNOMED","371808007","Recurrent angina status post percutaneous transluminal coronary angioplasty (disorder)","1","20210127"),
("angina","SNOMED","371810009","Recurrent angina status post coronary artery bypass graft (disorder)","1","20210127"),
("angina","SNOMED","371811008","Recurrent angina status post rotational atherectomy (disorder)","1","20210127"),
("angina","SNOMED","394659003","Acute coronary syndrome (disorder)","1","20210127"),
("angina","SNOMED","413844008","Chronic myocardial ischemia (disorder)","1","20210127"),
("angina","SNOMED","414545008","Ischemic heart disease (disorder)","1","20210127"),
("angina","SNOMED","414795007","Myocardial ischemia (disorder)","1","20210127"),
("angina","SNOMED","46109009","Subendocardial ischemia (disorder)","1","20210127"),
("angina","SNOMED","697976003","Microvascular ischemia of myocardium (disorder)","1","20210127"),
("angina","SNOMED","703214003","Silent coronary vasospastic disease (disorder)","1","20210127"),
("angina","SNOMED","713405002","Subacute ischemic heart disease (disorder)","1","20210127"),
("angina","SNOMED","87343002","Prinzmetal angina (disorder)","1","20210127"),
("unstable_angina","ICD10","I20.1","Unstable angina","",""),
("unstable_angina","SNOMED","15960061000119102","Unstable angina co-occurrent and due to coronary arteriosclerosis (disorder)","1","20210127"),
("unstable_angina","SNOMED","15960661000119107","Unstable angina co-occurrent and due to arteriosclerosis of coronary artery bypass graft (disorder)","1","20210127"),
("unstable_angina","SNOMED","19057007","Status anginosus (disorder)","1","20210127"),
("unstable_angina","SNOMED","25106000","Impending infarction (disorder)","1","20210127"),
("unstable_angina","SNOMED","4557003","Preinfarction syndrome (disorder)","1","20210127"),
("unstable_angina","SNOMED","712866001","Resting ischemia co-occurrent and due to ischemic heart disease (disorder)","1","20210127"),
("unstable_angina","SNOMED","315025001","Refractory angina (disorder)","1","20210127"),
("unstable_angina","SNOMED","35928006","Nocturnal angina (disorder)","1","20210127"),
("unstable_angina","SNOMED","371807002","Atypical angina (disorder)","1","20210127"),
("unstable_angina","SNOMED","394659003","Acute coronary syndrome (disorder)","1","20210127")

],
['name', 'terminology', 'code', 'term', 'code_type', 'RecordDate']   
).select("name","terminology","code","term")


crf_cad = crf_cad.union(codelist_angina).distinct()

display(crf_cad)

#3. Post Intervention Outcomes

**Only need the ICD10 codes for hospitalisation and death outcomes as these only come from HES and Deaths which use only ICD10**

##3.1 Hospitalisation with Heart Failure

HF codes as above - codes in the primary position only

##3.2 CVD Death

In [0]:
#from CCU004_01

# Fatal cardiovascular disease (ICD10)
codelist_fatal = spark.createDataFrame(
  [
    #hypertensive disease
    ('hypertension','ICD10',  'I10',   'Essential (primary) hypertension','',''),
    ('hypertension','ICD10',  'I11',   'Hypertensive heart disease','',''),
    ('hypertension','ICD10',  'I110',   'Hypertensive heart disease with (congestive) heart failure','',''),
    ('hypertension','ICD10',  'I119',   'Hypertensive heart disease without (congestive) heart failure','',''),
    ('hypertension','ICD10',  'I12',   'Hypertensive renal disease','',''),
    ('hypertension','ICD10',  'I129',   'Hypertensive renal disease without renal failure','',''),
    ('hypertension','ICD10',  'I13',   'Hypertensive heart and renal disease','',''),
    ('hypertension','ICD10',  'I130',   'Hypertensive heart and renal disease with (congestive) heart failure','',''),
    ('hypertension','ICD10',  'I131',   'Hypertensive heart and renal disease with renal failure','',''),
    ('hypertension','ICD10',  'I132',   'Hypertensive heart and renal disease with both (congestive) heart failure and renal failure','',''),
    ('hypertension','ICD10',  'I139',   'Hypertensive heart and renal disease, unspecified','',''),
    ('hypertension','ICD10',  'I15',   'Secondary hypertension','',''),
    ('hypertension','ICD10',  'I151',   'Hypertension secondary to other renal disorders','',''),
    ('hypertension','ICD10',  'I152',   'Hypertension secondary to endocrine disorders','',''),
    ('hypertension','ICD10',  'I158',   'Other secondary hypertension','',''),
    ('hypertension','ICD10',  'I159',   'Secondary hypertension, unspecified','',''),
  
    # ischaemic heart disease
    ('ischaemic_heart_disease','ICD10',  'I20',   'Angina pectoris','',''),
    ('ischaemic_heart_disease','ICD10',  'I200',   'Unstable angina','',''),
    ('ischaemic_heart_disease','ICD10',  'I201',   'Angina pectoris with documented spasm','',''),
    ('ischaemic_heart_disease','ICD10',  'I208',   'Other forms of angina pectoris','',''),
    ('ischaemic_heart_disease','ICD10',  'I209',   'Angina pectoris, unspecified','',''),
    ('ischaemic_heart_disease','ICD10',  'I21',   'Acute myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I210',   'Acute transmural myocardial infarction of anterior wall','',''),
    ('ischaemic_heart_disease','ICD10',  'I211',   'Acute transmural myocardial infarction of inferior wall','',''),
    ('ischaemic_heart_disease','ICD10',  'I212',   'Acute transmural myocardial infarction of other sites','',''),
    ('ischaemic_heart_disease','ICD10',  'I213',   'Acute transmural myocardial infarction of unspecified sites','',''),
    ('ischaemic_heart_disease','ICD10',  'I214',   'Acute subendocardial myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I219',   'Acute myocardial infarction, unspecified','',''),
    ('ischaemic_heart_disease','ICD10',  'I22',   'Subsequent myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I220',   'Subsequent myocardial infarction of anterior wall','',''),
    ('ischaemic_heart_disease','ICD10',  'I221',   'Subsequent myocardial infarction of inferior wall','',''),
    ('ischaemic_heart_disease','ICD10',  'I228',   'Subsequent myocardial infarction of other sites','',''),
    ('ischaemic_heart_disease','ICD10',  'I229',   'Subsequent myocardial infarction of unspecified site','',''),
    ('ischaemic_heart_disease','ICD10',  'I23',   'Certain current complications following acute myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I230',   'Haemopericardium as current complication following acute myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I231',   'Atrial septal defect as current complication following acute myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I232',   'Ventricular septal defect as current complication following acute myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I233',   'Rupture of cardiac wall without haemopericardium as current complication following acute myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I234',   'Rupture of chordae tendineae as current complication following acute myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I235',   'Rupture of papillary muscle as current complication following acute myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I236',   'Thrombosis of atrium, auricular appendage, and ventricle as current complications following acute myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I238',   'Other current complications following acute myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I24',   'Other acute ischaemic heart diseases','',''),
    ('ischaemic_heart_disease','ICD10',  'I240',   'Coronary thrombosis not resulting in myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I241',   'Dressler syndrome','',''),
    ('ischaemic_heart_disease','ICD10',  'I248',   'Other forms of acute ischaemic heart disease','',''),
    ('ischaemic_heart_disease','ICD10',  'I249',   'Acute ischaemic heart disease, unspecified','',''),
    ('ischaemic_heart_disease','ICD10',  'I25',    'Chronic ischemic heart disease','',''), #added by Fionna
    ('ischaemic_heart_disease','ICD10',  'I250',   'Atherosclerotic cardiovascular disease, so described','',''),
    ('ischaemic_heart_disease','ICD10',  'I251',   'Atherosclerotic heart disease','',''),
    ('ischaemic_heart_disease','ICD10',  'I252',   'Old myocardial infarction','',''),
    ('ischaemic_heart_disease','ICD10',  'I253',   'Aneurysm of heart','',''),
    ('ischaemic_heart_disease','ICD10',  'I254',   'Coronary artery aneurysm and dissection','',''),
    ('ischaemic_heart_disease','ICD10',  'I255',   'Ischaemic cardiomyopathy','',''),
    ('ischaemic_heart_disease','ICD10',  'I256',   'Silent myocardial ischaemia','',''),
    ('ischaemic_heart_disease','ICD10',  'I258',   'Other forms of chronic ischaemic heart disease','',''),
    ('ischaemic_heart_disease','ICD10',  'I259',   'Chronic ischaemic heart disease, unspecified','',''),
    
    # Arrythmias, heart failure
    ('arrythmias_heart_failure','ICD10',  'I46',   'Cardiac arrest','',''),
    ('arrythmias_heart_failure','ICD10',  'I460',   'Cardiac arrest with successful resuscitation','',''),
    ('arrythmias_heart_failure','ICD10',  'I461',   'Sudden cardiac death, so described','',''),
    ('arrythmias_heart_failure','ICD10',  'I47',   'Paroxysmal tachycardia','',''),
    ('arrythmias_heart_failure','ICD10',  'I470',   'Re-entry ventricular arrhythmia','',''),
    ('arrythmias_heart_failure','ICD10',  'I471',   'Supraventricular tachycardia','',''),
    ('arrythmias_heart_failure','ICD10',  'I472',   'Ventricular tachycardia','',''),
    ('arrythmias_heart_failure','ICD10',  'I479',   'Paroxysmal tachycardia, unspecified','',''),
    ('arrythmias_heart_failure','ICD10',  'I48',   'Atrial fibrillation and flutter','',''),
    ('arrythmias_heart_failure','ICD10',  'I480',   'Paroxysmal atrial fibrillation','',''),   
    ('arrythmias_heart_failure','ICD10',  'I481',   'Persistent atrial fibrillation','',''),
    ('arrythmias_heart_failure','ICD10',  'I482',   'Chronic atrial fibrillation','',''),
    ('arrythmias_heart_failure','ICD10',  'I483',   'Typical atrial flutter','',''),
    ('arrythmias_heart_failure','ICD10',  'I484',   'Atypical atrial flutter','',''),
    ('arrythmias_heart_failure','ICD10',  'I489',   'Atrial fibrillation and atrial flutter, unspecified','',''),
    ('arrythmias_heart_failure','ICD10',  'I49',   'Other cardiac arrhythmias','',''),
    ('arrythmias_heart_failure','ICD10',  'I491',   'Atrial premature depolarization','',''),
    ('arrythmias_heart_failure','ICD10',  'I492',   'Junctional premature depolarization','',''),
    ('arrythmias_heart_failure','ICD10',  'I494',   'Ventricular premature depolarization','',''),
    ('arrythmias_heart_failure','ICD10',  'I495',   'Sick sinus syndrome','',''),  
    ('arrythmias_heart_failure','ICD10',  'I498',   'Other specified cardiac arrhythmias','',''),
    ('arrythmias_heart_failure','ICD10',  'I499',   'Cardiac arrhythmia, unspecified','',''),
    ('arrythmias_heart_failure','ICD10',  'I50',   'Heart failure','',''),
    ('arrythmias_heart_failure','ICD10',  'I500',   'Congestive heart failure','',''),
    ('arrythmias_heart_failure','ICD10',  'I501',   'Left ventricular failure','',''),
    ('arrythmias_heart_failure','ICD10',  'I509',   'Heart failure, unspecified','',''),
   ('arrythmias_heart_failure','ICD10',  'I51',   'Complications and ill-defined descriptions of heart disease','',''), #added back in by Fionna
    ('arrythmias_heart_failure','ICD10',  'I510',   'Cardiac septal defect, acquired','',''),
    ('arrythmias_heart_failure','ICD10',  'I511',   'Rupture of chordae tendineae, not elsewhere classified','',''),
    ('arrythmias_heart_failure','ICD10',  'I512',   'Rupture of papillary muscle, not elsewhere classified','',''),
    ('arrythmias_heart_failure','ICD10',  'I513',   'Intracardiac thrombosis, not elsewhere classified','',''),
  ('arrythmias_heart_failure','ICD10',  'I514',   'Myocarditis, unspecified','',''),
    ('arrythmias_heart_failure','ICD10',  'I515',   'Myocardial degeneration','',''),   
    ('arrythmias_heart_failure','ICD10',  'I516',   'Cardiovascular disease, unspecified','',''),
    ('arrythmias_heart_failure','ICD10',  'I517',   'Cardiomegaly','',''),
    ('arrythmias_heart_failure','ICD10',  'I518',   'Other ill-defined heart diseases','',''),
    ('arrythmias_heart_failure','ICD10',  'I519',   'Heart disease, unspecified','',''),
    ('arrythmias_heart_failure','ICD10',  'I52',   'Other heart disorders in diseases classified elsewhere','',''),
    ('arrythmias_heart_failure','ICD10',  'I520',   'Other heart disorders in bacterial diseases classified elsewhere','',''),
    ('arrythmias_heart_failure','ICD10',  'I521',   'Other heart disorders in other infectious and parasitic diseases classified elsewhere','',''),
    ('arrythmias_heart_failure','ICD10',  'I528',   'Other heart disorders in other diseases classified elsewhere','',''),

    # Cerebrovascular disease
    
    #excluded in CCU004_01
    ('cerebrovascular_disease','ICD10',  'I60',   'Subarachnoid haemorrhage','',''),
    ('cerebrovascular_disease','ICD10',  'I600',  'Subarachnoid haemorrhage from carotid siphon and bifurcation','',''),
    ('cerebrovascular_disease','ICD10',  'I601',  'Subarachnoid haemorrhage from middle cerebral artery','',''),
    ('cerebrovascular_disease','ICD10',  'I602',  'Subarachnoid haemorrhage from anterior communicating artery','',''),
    ('cerebrovascular_disease','ICD10',  'I603',  'Subarachnoid haemorrhage from posterior communicating artery','',''),
    ('cerebrovascular_disease','ICD10',  'I604',  'Subarachnoid haemorrhage from basilar artery','',''),
    ('cerebrovascular_disease','ICD10',  'I605',  'Subarachnoid haemorrhage from vertebral artery','',''),
    ('cerebrovascular_disease','ICD10',  'I606',  'Subarachnoid haemorrhage from other intracranial arteries','',''),
    ('cerebrovascular_disease','ICD10',  'I607',  'Subarachnoid haemorrhage from intracranial artery, unspecified','',''),
    ('cerebrovascular_disease','ICD10',  'I608',  'Other subarachnoid haemorrhage','',''),
    ('cerebrovascular_disease','ICD10',  'I609',  'Subarachnoid haemorrhage, unspecified','',''),



    ('cerebrovascular_disease','ICD10',  'I61',   'Intracerebral haemorrhage','',''),
    ('cerebrovascular_disease','ICD10',  'I610',  'Intracerebral haemorrhage in hemisphere, subcortical','',''),
    ('cerebrovascular_disease','ICD10',  'I611',  'Intracerebral haemorrhage in hemisphere, cortical','',''),
    ('cerebrovascular_disease','ICD10',  'I612',  'Intracerebral haemorrhage in hemisphere, unspecified','',''),
    ('cerebrovascular_disease','ICD10',  'I613',  'Intracerebral haemorrhage in brain stem','',''),
    ('cerebrovascular_disease','ICD10',  'I614',  'Intracerebral haemorrhage in cerebellum','',''),
    ('cerebrovascular_disease','ICD10',  'I615',  'Intracerebral haemorrhage, intraventricular','',''),
    ('cerebrovascular_disease','ICD10',  'I616',  'Intracerebral haemorrhage, multiple localized','',''),
    ('cerebrovascular_disease','ICD10',  'I618',  'Other intracerebral haemorrhage','',''),
    ('cerebrovascular_disease','ICD10',  'I619',  'Intracerebral haemorrhage, unspecified','',''),


     #excluded in CCU004_01
    ('cerebrovascular_disease','ICD10',  'I62',   'Other nontraumatic intracranial haemorrhage','',''),
    ('cerebrovascular_disease','ICD10',  'I620',  'Subdural haemorrhage (acute)(nontraumatic)','',''),
    ('cerebrovascular_disease','ICD10',  'I621',  'Nontraumatic extradural haemorrhage','',''),
    ('cerebrovascular_disease','ICD10',  'I629',  'Intracranial haemorrhage (nontraumatic), unspecified','',''),


    ('cerebrovascular_disease','ICD10',  'I63',   'Cerebral infarction','',''),
    ('cerebrovascular_disease','ICD10',  'I630',  'Cerebral infarction due to thrombosis of precerebral arteries','',''),
    ('cerebrovascular_disease','ICD10',  'I631',  'Cerebral infarction due to embolism of precerebral arteries','',''),
    ('cerebrovascular_disease','ICD10',  'I632',  'Cerebral infarction due to unspecified occlusion or stenosis of precerebral arteries','',''),
    ('cerebrovascular_disease','ICD10',  'I633',  'Cerebral infarction due to thrombosis of cerebral arteries','',''),
    ('cerebrovascular_disease','ICD10',  'I634',  'Cerebral infarction due to embolism of cerebral arteries','',''),
    ('cerebrovascular_disease','ICD10',  'I635',  'Cerebral infarction due to unspecified occlusion or stenosis of cerebral arteries','',''),
    ('cerebrovascular_disease','ICD10',  'I636',  'Cerebral infarction due to cerebral venous thrombosis, nonpyogenic','',''),
    ('cerebrovascular_disease','ICD10',  'I638',  'Other cerebral infarction','',''),
    ('cerebrovascular_disease','ICD10',  'I639',  'Cerebral infarction, unspecified','',''),
    ('cerebrovascular_disease','ICD10',  'I64X',  'Stroke, not specified as haemorrhage or infarction','',''),
    ('cerebrovascular_disease','ICD10',  'I65',   'Occlusion and stenosis of precerebral arteries, not resulting in cerebral infarction','',''),
    ('cerebrovascular_disease','ICD10',  'I650',  'Occlusion and stenosis of vertebral artery','',''),
    ('cerebrovascular_disease','ICD10',  'I651',  'Occlusion and stenosis of basilar artery','',''),
    ('cerebrovascular_disease','ICD10',  'I652',  'Occlusion and stenosis of carotid artery','',''),
    ('cerebrovascular_disease','ICD10',  'I653',  'Occlusion and stenosis of multiple and bilateral precerebral arteries','',''),
    ('cerebrovascular_disease','ICD10',  'I658',  'Occlusion and stenosis of other precerebral artery','',''),
    ('cerebrovascular_disease','ICD10',  'I659',  'Occlusion and stenosis of unspecified precerebral artery','',''),
    ('cerebrovascular_disease','ICD10',  'I66',   'Occlusion and stenosis of cerebral arteries, not resulting in cerebral infarction','',''),
    ('cerebrovascular_disease','ICD10',  'I660',  'Occlusion and stenosis of middle cerebral artery','',''),
    ('cerebrovascular_disease','ICD10',  'I661',  'Occlusion and stenosis of anterior cerebral artery','',''),
    ('cerebrovascular_disease','ICD10',  'I662',  'Occlusion and stenosis of posterior cerebral artery','',''),
    ('cerebrovascular_disease','ICD10',  'I663',  'Occlusion and stenosis of cerebellar arteries','',''),
    ('cerebrovascular_disease','ICD10',  'I664',  'Occlusion and stenosis of multiple and bilateral cerebral arteries','',''),
    ('cerebrovascular_disease','ICD10',  'I668',  'Occlusion and stenosis of other cerebral artery','',''),
    ('cerebrovascular_disease','ICD10',  'I669',  'Occlusion and stenosis of unspecified cerebral artery','',''),

    ('cerebrovascular_disease','ICD10',  'I67',   'Other cerebrovascular diseases','',''), #prev excluded

    ('cerebrovascular_disease','ICD10',  'I670',  'Dissection of cerebral arteries, nonruptured','',''),

    ('cerebrovascular_disease','ICD10',  'I671',  'Cerebral aneurysm, nonruptured','',''), #prev excluded

    ('cerebrovascular_disease','ICD10',  'I672',  'Cerebral atherosclerosis','',''),
    ('cerebrovascular_disease','ICD10',  'I673',  'Progressive vascular leukoencephalopathy','',''),
    ('cerebrovascular_disease','ICD10',  'I674',  'Hypertensive encephalopathy','',''),

    ('cerebrovascular_disease','ICD10',  'I675',  'Moyamoya disease','',''), #prev excluded

    ('cerebrovascular_disease','ICD10',  'I676',  'Nonpyogenic thrombosis of intracranial venous system','',''),
    ('cerebrovascular_disease','ICD10',  'I677',  'Cerebral arteritis, not elsewhere classified','',''),
    ('cerebrovascular_disease','ICD10',  'I678',  'Other specified cerebrovascular diseases','',''),
    ('cerebrovascular_disease','ICD10',  'I679',  'Cerebrovascular disease, unspecified','',''),

    ('cerebrovascular_disease','ICD10',  'I68',   'Cerebrovascular disorders in diseases classified elsewhere','',''), #prev excluded

    ('cerebrovascular_disease','ICD10',  'I680',  'Cerebral amyloid angiopathy','',''),
    ('cerebrovascular_disease','ICD10',  'I681',  'Cerebral arteritis in infectious and parasitic diseases classified elsewhere','',''),

    ('cerebrovascular_disease','ICD10',  'I682',  'Cerebral arteritis in other diseases classified elsewhere','',''), #prev excluded

    ('cerebrovascular_disease','ICD10',  'I688',  'Other cerebrovascular disorders in diseases classified elsewhere','',''),
    ('cerebrovascular_disease','ICD10',  'I69',   'Sequelae of cerebrovascular disease','',''),
    ('cerebrovascular_disease','ICD10',  'I690',  'Sequelae of subarachnoid haemorrhage','',''),
    ('cerebrovascular_disease','ICD10',  'I691',  'Sequelae of intracerebral haemorrhage','',''),
    ('cerebrovascular_disease','ICD10',  'I692',  'Sequelae of other nontraumatic intracranial haemorrhage','',''),
    ('cerebrovascular_disease','ICD10',  'I693',  'Sequelae of cerebral infarction','',''),
    ('cerebrovascular_disease','ICD10',  'I694',  'Sequelae of stroke, not specified as haemorrhage or infarction','',''),
    ('cerebrovascular_disease','ICD10',  'I698',  'Sequelae of other and unspecified cerebrovascular diseases','',''),



   # Athelerosclerosis/AAA
    ('atherosclerosis','ICD10',  'I70',   'Atherosclerosis','',''),
    ('atherosclerosis','ICD10',  'I700',   'Atherosclerosis of aorta','',''),   
    ('atherosclerosis','ICD10',  'I7000',   'Atherosclerosis of aorta','',''),
    ('atherosclerosis','ICD10',  'I7001',   'Atherosclerosis of aorta','',''),   
    ('atherosclerosis','ICD10',  'I701',   'Atherosclerosis of renal artery','',''),
    ('atherosclerosis','ICD10',  'I7010',   'Atherosclerosis of renal artery','',''),   
    ('atherosclerosis','ICD10',  'I7011',   'Atherosclerosis of renal artery','',''),
    ('atherosclerosis','ICD10',  'I702',   'Atherosclerosis of arteries of extremities','',''),   
    ('atherosclerosis','ICD10',  'I7020',   'Atherosclerosis of arteries of extremities','',''),
    ('atherosclerosis','ICD10',  'I7021',   'Atherosclerosis of arteries of extremities','',''),   
    ('atherosclerosis','ICD10',  'I708',   'Atherosclerosis of other arteries','',''),
    ('atherosclerosis','ICD10',  'I7080',   'Atherosclerosis of other arteries','',''),
    ('atherosclerosis','ICD10',  'I7081',   'Atherosclerosis of other arteries','',''),   
    ('atherosclerosis','ICD10',  'I709',   'Generalized and unspecified atherosclerosis','',''),
    ('atherosclerosis','ICD10',  'I7090',   'Generalized and unspecified atherosclerosis','',''),   
    ('atherosclerosis','ICD10',  'I7091',   'Generalized and unspecified atherosclerosis','',''),
    ('atherosclerosis','ICD10',  'I71',   'Aortic aneurysm and dissection','',''),
    ('atherosclerosis','ICD10',  'I710',   'Dissection of aorta [any part]','',''),
    ('atherosclerosis','ICD10',  'I711',   'Thoracic aortic aneurysm, ruptured','',''),   
    ('atherosclerosis','ICD10',  'I712',   'Thoracic aortic aneurysm, without mention of rupture','',''),
    ('atherosclerosis','ICD10',  'I713',   'Abdominal aortic aneurysm, ruptured','',''),   
    ('atherosclerosis','ICD10',  'I714',   'Abdominal aortic aneurysm, without mention of rupture','',''),
    ('atherosclerosis','ICD10',  'I715',   'Thoracoabdominal aortic aneurysm, ruptured','',''),
    ('atherosclerosis','ICD10',  'I716',   'Thoracoabdominal aortic aneurysm, without mention of rupture','',''),   
    ('atherosclerosis','ICD10',  'I718',   'Aortic aneurysm of unspecified site, ruptured','',''),
    ('atherosclerosis','ICD10',  'I719',   'Aortic aneurysm of unspecified site, without mention of rupture','',''),  
    ('atherosclerosis','ICD10',  'I72',   'Other aneurysm and dissection','',''),   
    ('atherosclerosis','ICD10',  'I720',   'Aneurysm and dissection of carotid artery','',''),
    ('atherosclerosis','ICD10',  'I721',   'Aneurysm and dissection of artery of upper extremity','',''),  
    ('atherosclerosis','ICD10',  'I722',   'Aneurysm and dissection of renal artery','',''),   
    ('atherosclerosis','ICD10',  'I724',   'Aneurysm and dissection of artery of lower extremity','',''),
    ('atherosclerosis','ICD10',  'I725',   'Aneurysm and dissection of other precerebral arteries','',''),  
    ('atherosclerosis','ICD10',  'I726',   'Aneurysm and dissection of vertebral artery','',''),   
    ('atherosclerosis','ICD10',  'I728',   'Aneurysm and dissection of other specified arteries','',''),
    ('atherosclerosis','ICD10',  'I729',   'Aneurysm and dissection of unspecified site','',''),  
    ('atherosclerosis','ICD10',  'I73',   'Other peripheral vascular diseases','',''),   
    ('atherosclerosis','ICD10',  'I730',   'Raynaud syndrome','',''),
    ('atherosclerosis','ICD10',  'I731',   'Thromboangiitis obliterans [Buerger]','',''),  
    ('atherosclerosis','ICD10',  'I738',   'Other specified peripheral vascular diseases','',''),
    ('atherosclerosis','ICD10',  'I739',   'Peripheral vascular disease, unspecified','',''), 
        
    # Sudden death and death within 24h of symptom onset
    ('sudden_death','ICD10',  'R96',   'Other sudden death, cause unknown','',''),   
    ('sudden_death','ICD10',  'R960',   'Instantaneous death','',''),
    ('sudden_death','ICD10',  'R961',   'Death occurring less than 24 hours from onset of symptoms, not otherwise explained','',''),  
    
  ],
  
  ['name', 'terminology', 'code', 'term', 'code_type', 'RecordDate']  
).select("name","terminology","code","term")


pio_fatal = codelist_fatal.distinct()

display(pio_fatal)

# 4. Export

In [0]:
codelist_all = (
    comorbidities_hf.withColumn("codelist", f.lit("Comorbidities - Heart Failure"))
.union(comorbidities_copd.withColumn("codelist", f.lit("Comorbidities - COPD")))
.union(comorbidities_asthma.withColumn("codelist", f.lit("Comorbidities - Asthma")))
.union(comorbidities_ckd.withColumn("codelist", f.lit("Comorbidities - Renal Failure (CKD)")))
.union(comorbidities_cd.withColumn("codelist", f.lit("Comorbidities - Cerebrovascular Disease")))
.union(comorbidities_pad.withColumn("codelist", f.lit("Comorbidities - Peropheral Arterial Disease")))

.union(crf_hypertension.withColumn("codelist", f.lit("Cardiovascular Risk Factors - Hypertension")))
.union(crf_diabetes.withColumn("codelist", f.lit("Cardiovascular Risk Factors - Diabetes")))
.union(crf_cad.withColumn("codelist", f.lit("Cardiovascular Risk Factors - Coronary Artery Disease")))

.union(codelist_fatal.withColumn("codelist", f.lit("Post Intervention Outcomes - CVD Death")))
)

In [0]:
display(codelist_all.select("codelist","name","terminology","code","term").orderBy("codelist","name","terminology","code"))

# 5. Save

In [0]:
save_table(codelist_all, out_name=f'{proj}_out_codelist_covariates')